In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.master('local[2]').getOrCreate()


In [14]:


globalsocialnetworks = spark.read.csv("hdfs://hdfs-nn:9000/user/Projeto TABD/Bronze/MostPopularSocialNetworksWorldwideAsOfJanuary2021RankedByNumberOfActiveUsers.csv", 
                                                 header=True, 
                                                 sep=";")

In [15]:
globalsocialnetworks.printSchema()
globalsocialnetworks.show()
globalsocialnetworks.toPandas()

root
 |-- Global social networks ranked by number of users 2021: string (nullable = true)
 |-- _c1: string (nullable = true)

+-----------------------------------------------------+-----+
|Global social networks ranked by number of users 2021|  _c1|
+-----------------------------------------------------+-----+
|                                 Most popular soci...| null|
|                                                 null| null|
|                                             Facebook|2 740|
|                                              YouTube|2 291|
|                                             WhatsApp|2 000|
|                                  Facebook Messenger*|1 300|
|                                            Instagram|1 221|
|                                      Weixin / WeChat|1 213|
|                                               TikTok|  689|
|                                                   QQ|  617|
|                                               Douyin|  600|
|     

,Global social networks ranked by number of users 2021,_c1
0,Most popular social networks worldwide as of J...,None
1,None,None
2,Facebook,2 740
3,YouTube,2 291
4,WhatsApp,2 000
5,Facebook Messenger*,1 300
6,Instagram,1 221
7,Weixin / WeChat,1 213
8,TikTok,689
9,QQ,617


In [16]:
globalsocialnetworks = globalsocialnetworks.selectExpr("`Global social networks ranked by number of users 2021` as social_network", 
                                                                         "_c1 as users",
                                                                         "'2021' as year"
                                                                        )

In [17]:
globalsocialnetworks.toPandas()

,social_network,users,year
0,Most popular social networks worldwide as of J...,None,2021
1,None,None,2021
2,Facebook,2 740,2021
3,YouTube,2 291,2021
4,WhatsApp,2 000,2021
5,Facebook Messenger*,1 300,2021
6,Instagram,1 221,2021
7,Weixin / WeChat,1 213,2021
8,TikTok,689,2021
9,QQ,617,2021


In [18]:
# Substitui o valor  por None para depois remover a linha toda
globalsocialnetworks = globalsocialnetworks.withColumn(
    "social_network",
    when(
        (col("social_network") == "String"), 
        None
    ).otherwise(concat(lit(""), col("social_network"))))

globalsocialnetworks = globalsocialnetworks.withColumn(
    "users",
    when(
        (col("users") == "String"), 
        None
    ).otherwise(concat(lit(""), col("users"))))



globalsocialnetworks = globalsocialnetworks.dropna(how='any')

In [19]:
globalsocialnetworks.toPandas()

,social_network,users,year
0,Facebook,2 740,2021
1,YouTube,2 291,2021
2,WhatsApp,2 000,2021
3,Facebook Messenger*,1 300,2021
4,Instagram,1 221,2021
5,Weixin / WeChat,1 213,2021
6,TikTok,689,2021
7,QQ,617,2021
8,Douyin,600,2021
9,Sina Weibo,511,2021


In [20]:
globalsocialnetworks = globalsocialnetworks.withColumn(
    "social_network",
    f.regexp_replace(col("social_network"),"[*]", ""))

globalsocialnetworks = globalsocialnetworks.withColumn(
    "users",
    f.regexp_replace(col("users")," ", ""))

In [21]:
globalsocialnetworks.toPandas()

,social_network,users,year
0,Facebook,2740,2021
1,YouTube,2291,2021
2,WhatsApp,2000,2021
3,Facebook Messenger,1300,2021
4,Instagram,1221,2021
5,Weixin / WeChat,1213,2021
6,TikTok,689,2021
7,QQ,617,2021
8,Douyin,600,2021
9,Sina Weibo,511,2021


In [22]:
globalsocialnetworks = globalsocialnetworks.withColumn(
    "users",
    globalsocialnetworks["users"].cast(IntegerType()))

globalsocialnetworks = globalsocialnetworks.withColumn(
    "year",
    globalsocialnetworks["year"].cast(IntegerType()))

In [23]:
globalsocialnetworks.printSchema()

root
 |-- social_network: string (nullable = true)
 |-- users: integer (nullable = true)
 |-- year: integer (nullable = true)



In [24]:
globalsocialnetworks \
    .write \
    .format("parquet") \
    .mode("append") \
    .save("hdfs://hdfs-nn:9000/warehouse/tabd.db/GlobalSocialNetworks/")